In [1]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from google.cloud import language_v1
import boto3
from dotenv import load_dotenv
import os

load_dotenv()
IBM_API_KEY = os.getenv("IBM_API_KEY")
IBM_URL = os.getenv("IBM_URL")

GOOGLE_API_CREDENTIALS_PATH = os.getenv("GOOGLE_API_CREDENTIALS_PATH")

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
AWS_REGION = os.getenv("AWS_REGION")


/Users/sonamjain/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
text = "Tesla's Cybertruck preorders skyrocket after unveiling."

In [3]:
# Initialize IBM Watson NLU
def analyze_ibm():
    authenticator = IAMAuthenticator(IBM_API_KEY)
    nlu = NaturalLanguageUnderstandingV1(
        version="2021-08-01",
        authenticator=authenticator
    )
    nlu.set_service_url(IBM_URL)

    response = nlu.analyze(
        text=text,
        features=Features(keywords=KeywordsOptions(limit=5))
    ).get_result()
    return response

# Initialize Google Cloud NLP
def analyze_google():
    client = language_v1.LanguageServiceClient.from_service_account_json(GOOGLE_API_CREDENTIALS_PATH)

    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    response = client.analyze_entities(document=document)
    return response

# Initialize AWS Comprehend
def analyze_aws():
    client = boto3.client(
        "comprehend",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )

    response = client.detect_entities(
        Text=text,
        LanguageCode="en"
    )
    return response

In [4]:
# Measure response time for a single request
def measure_response_time(func, tool_name):
    start_time = time.time()
    func()
    end_time = time.time()
    response_time = end_time - start_time
    print(f"{tool_name} Response Time: {response_time:.2f} seconds")
    return response_time

# Simulate parallel requests
def simulate_parallel_requests(func, tool_name, num_requests=1000, max_workers=10):
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(func) for _ in range(num_requests)]
        for future in as_completed(futures):
            pass
    end_time = time.time()
    total_time = end_time - start_time
    avg_response_time = total_time / num_requests
    print(f"{tool_name} Average Response Time for {num_requests} Parallel Requests: {avg_response_time:.2f} seconds")
    return avg_response_time

In [5]:
ibm_response_time = measure_response_time(analyze_ibm, "IBM Watson NLU")
ibm_avg_parallel_time = simulate_parallel_requests(analyze_ibm, "IBM Watson NLU", num_requests=1000, max_workers=20)

IBM Watson NLU Response Time: 1.09 seconds
IBM Watson NLU Average Response Time for 1000 Parallel Requests: 0.04 seconds


In [6]:
google_response_time = measure_response_time(analyze_google, "Google Cloud NLP")
google_avg_parallel_time = simulate_parallel_requests(analyze_google, "Google Cloud NLP", num_requests=1000, max_workers=20)

Google Cloud NLP Response Time: 0.30 seconds
Google Cloud NLP Average Response Time for 1000 Parallel Requests: 0.04 seconds


In [7]:
aws_response_time = measure_response_time(analyze_aws, "AWS Comprehend")
aws_avg_parallel_time = simulate_parallel_requests(analyze_aws, "AWS Comprehend", num_requests=1000, max_workers=20)

AWS Comprehend Response Time: 0.35 seconds
AWS Comprehend Average Response Time for 1000 Parallel Requests: 0.02 seconds


In [8]:
print(f"IBM Watson NLU: Single Request: {ibm_response_time:.2f}s, Parallel Avg: {ibm_avg_parallel_time:.2f}s")
print(f"Google Cloud NLP: Single Request: {google_response_time:.2f}s, Parallel Avg: {google_avg_parallel_time:.2f}s")
print(f"AWS Comprehend: Single Request: {aws_response_time:.2f}s, Parallel Avg: {aws_avg_parallel_time:.2f}s")

IBM Watson NLU: Single Request: 1.09s, Parallel Avg: 0.04s
Google Cloud NLP: Single Request: 0.30s, Parallel Avg: 0.04s
AWS Comprehend: Single Request: 0.35s, Parallel Avg: 0.02s
